# Predictive Maintenance

This notebook is part of [*Hands-on Machine Learning for IoT*](https://github.com/pablodecm/datalab_ml_iot) tutorial by Pablo de Castro

## Tools

This notebook will use the following Python 3
libraries for data analytics and machine learning:
- pandas
- numpy
- matplotlib
- scikit-learn
- keras/tensorflow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Dataset

The dataset that will be used, which was published by NASA [[3](#References)],
consist on simulated turbojet engine degradation
under different combinations of operational conditions.

<div align="center">
  <img src="images/airbus-turbofan.jpg" height="50%" style="max-width: 50%">
</div>


In [ ]:
!mkdir data
!wget https://ti.arc.nasa.gov/c/6/ -O data/CMAPSSData.zip
!unzip -o data/CMAPSSData.zip -d data
!ls -lrth data/*

In [ ]:
cat data/readme.txt

In [ ]:
# train and test data are simple space separated values
!head -5 data/train_FD001.txt

In [ ]:
# the test truth is given as the number of steps to failure
# for each engine run in the test set (100 in total)
!head -5 data/RUL_FD001.txt

In [ ]:
# load data (only gonna use FD001 dataset)
train_df = pd.read_csv('data/train_FD001.txt', sep=" ", header=None)
test_df  = pd.read_csv('data/test_FD001.txt', sep=" ", header=None)
print("train shape: ", train_df.shape, "test shape: ", test_df.shape)
# lets have a look at basic descriptive statistics
train_df.describe()

In [ ]:
# we will remove columns 26 and 27 because of the NaNs
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
print("train shape: ", train_df.shape, "test shape: ", test_df.shape)

In [ ]:
# the files did not contain headers
# we can create them based on the documentation
target_var = ['target_RUL']
index_columns_names =  ["UnitNumber","Cycle"]
op_settings_columns = ["Op_Setting_"+str(i) for i in range(1,4)]
sensor_columns =["Sensor_"+str(i) for i in range(1,22)]
column_names = index_columns_names + op_settings_columns + sensor_columns
print(column_names)

In [ ]:
# name columns
train_df.columns = column_names
test_df.columns = column_names

# now the dataset looks better, e.g. the first unit
train_df[train_df.UnitNumber == 1].head(5)

### Remaining Useful Life (RUL)

The training data consists time-series for the engine sensors
for each cycle (i.e. timestep) until failure which happens
after the last time step.

Thus, the Remaining Useful Life (RUL), i.e. time until the
engine breaks, can be calculated based on the maximum cycle
of each unit present in the training set.


In [ ]:
# find the last cycle per unit number
max_cycle = train_df.groupby('UnitNumber')['Cycle'].max().reset_index()
max_cycle.columns = ['UnitNumber', 'MaxOfCycle']
# merge the max cycle back into the original frame
train_df = train_df.merge(max_cycle, left_on='UnitNumber', right_on='UnitNumber', how='inner')
# calculate RUL for each row
target_RUL = train_df["MaxOfCycle"] - train_df["Cycle"]
# add columns and remove MaxOfCycle
train_df["target_RUL"] = target_RUL
train_df = train_df.drop("MaxOfCycle", axis=1)
# check that it worked for unit 1
train_df[train_df.UnitNumber == 1].head(5)

The test data does not correspond to running until failure
as discussed in the dataset documentation, the RUL at the last
step is instead provided on an additional file.

In [ ]:
# get truth RUL
truth_df = pd.read_csv('data/RUL_FD001.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)
# UnitNumber based
truth_df.columns = ["RUL_after_last"]
truth_df['UnitNumber'] =  truth_df.index + 1
# find the last cycle per unit number in test set
max_cycle = test_df.groupby('UnitNumber')['Cycle'].max().reset_index()
max_cycle.columns = ['UnitNumber', 'MaxOfCycle']
max_cycle['MaxOfCycle'] = max_cycle['MaxOfCycle'] + truth_df["RUL_after_last"]
# merge the max cycle back into the original frame
test_df = test_df.merge(max_cycle, left_on='UnitNumber', right_on='UnitNumber', how='inner')
# calculate RUL for each row
target_RUL = test_df["MaxOfCycle"] - test_df["Cycle"]
# add columns and remove MaxOfCycle
test_df["target_RUL"] = target_RUL
test_df = test_df.drop("MaxOfCycle", axis=1)
# check that it worked for unit 1
test_df[test_df.UnitNumber == 1].head(5)

### Defining the Problem

### Feature Exploration

Independently on the chosen problem, it is always recommended
to interactively explore the variables to be considered
in the predictive modelling problem.

It is important to not only consider the data available
in the training set but also that expected in the test
or production environment. Alternatively we can
incur in:
- **target leakage**: use information that has predictive power
as input of the model during training but will no be available
in production or for the real data.
- **domain mismatch**: if the training and test/production data
are different the trained models might not perform well in
the real word scenario.

In [ ]:
# we will consider all features except the UnitNumber and the target
basic_features = train_df.columns.difference(["UnitNumber","target_RUL"])
print("basic_features: ",basic_features)

### Feature Transformations and Engineering

Another important step, particularly when dealing with
most techniques other than Deep Learning, is *feature
preprocessing/scaling* and *feature engineering*.

Feature preprocessing/scaling can facilitate model training by
scaling the features to dimensionless values based on the properties
of the dataset.

Feature engineering, the definition of new variables based
on those available, is particularly important for time-series
data when we want to
make a prediction for each timestep as is the case for all the
problems considered in this notebook.




In [ ]:
from sklearn import preprocessing

# we will use the Standard Scaler
scaler = preprocessing.StandardScaler()

features = basic_features
X_unscaled = train_df[features].astype('float64')
X = pd.DataFrame(scaler.fit_transform(X_unscaled),
                 columns = features,
                 index = train_df.index)
y = train_df["target_RUL"]

X.describe()

In [ ]:
X_test_unscaled = test_df[features].astype('float64')
X_test = pd.DataFrame(scaler.transform(X_test_unscaled),
                      columns = features,
                      index = test_df.index)
y_test = test_df["target_RUL"]

X_test.describe()

### RUL Prediction as a Regression Task

**How many more cycles an in-service engine will last before it fails?**

The task of predicting the Remaining Useful Life (RUL)
can be casted as a regression task in the context of machine
learning. RUL can also be referred as Time to Failure (TTF).


The goal in a regression problem is to find a function
$f_R(\boldsymbol{x})$ that approximates the true target $y$. The
target in this problem will be the RUL, while $\boldsymbol{x}$
will be the input features (e.g. sensor readings).

To measure the goodness of our regression function, we need
to define and score function, which will be also the loss
function for some of the machine learning techniques considered.
We will be considering the mean squared error:
$$ \textrm{MSE} = \sum_{i=1}^{n} (y - f_R(\boldsymbol{x}))^2$$
which is one of the most common losses used for regression,
depending on the problem alternative loss definitions might
be more beneficial.

#### Baseline and Feature Importance

Sometimes is useful to train a simple model for the task at hand to
get a baseline performance. A random forest has the advantage that
it can also provide a list of the relative importance of the
different features.

In [ ]:
from sklearn import ensemble

rf = ensemble.RandomForestRegressor()
simple_rf = ensemble.RandomForestRegressor(n_estimators = 200, max_depth = 15)
simple_rf.fit(X, y)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = simple_rf.predict(X)
print("[Train] Simple RF Mean Squared Error: ", mean_squared_error(y, y_pred))
print("[Train] Simple RF Mean Absolute Error: ", mean_absolute_error(y, y_pred))
print("[Train] Simple RF r-squared: ", r2_score(y, y_pred))

In [ ]:
y_test_pred = simple_rf.predict(X_test)
print("[Test] Simple RF Mean Squared Error: ", mean_squared_error(y_test, y_test_pred))
print("[Test] Simple RF Mean Absolute Error: ", mean_absolute_error(y_test, y_test_pred))
print("[Test] Simple RF r-squared: ", r2_score(y_test, y_test_pred))

In [ ]:
# graph feature importance
import matplotlib.pyplot as plt
importances = simple_rf.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X.columns    
f, ax = plt.subplots(figsize=(11, 9))
plt.title("Feature ranking", fontsize = 20)
plt.bar(range(X.shape[1]), importances[indices], color="b", align="center")
plt.xticks(range(X.shape[1]), indices) #feature_names, rotation='vertical')
plt.xlim([-1, X.shape[1]])
plt.ylabel("importance", fontsize = 18)
plt.xlabel("index of the feature", fontsize = 18)
plt.show()
# list feature importance
important_features = pd.Series(data=single_rf.feature_importances_,index=X.columns)
important_features.sort_values(ascending=False,inplace=True)
print(important_features.head(10))

#### Cross Validation and Hyper-Parameters

By means of cross validation and hyper-parameter search, we
can try to obtain a better regression model based on RandomForest.

In [ ]:
from sklearn.model_selection import GroupKFold, GridSearchCV

rf = ensemble.RandomForestRegressor(n_estimators=100)

# to avoid having same UnitNumber in both sets
cv = GroupKFold(5)

param_grid = { "min_samples_leaf" : [2, 10, 25, 50, 100],
               "max_depth" : [7, 8, 9, 10, 11, 12]}

optimized_rf = GridSearchCV(estimator=rf,
                            cv = cv,
                            param_grid=param_grid,
                            scoring='neg_mean_squared_error',
                            verbose = 1,
                            n_jobs = -1)

optimized_rf.fit(X, y, groups = train_df.UnitNumber)

In [ ]:
y_test_pred = optimized_rf.predict(X_test)
print("[Test] Optimized RF Mean Squared Error: ", mean_squared_error(y_test, y_test_pred))
print("[Test] Optimized RF Mean Absolute Error: ", mean_absolute_error(y_test, y_test_pred))
print("[Test] Optimized RF r-squared: ", r2_score(y_test, y_test_pred))

#### Other Models: Gradient Boosting



#### Exercise: Train Another Cross Validated Model

Based on the previous examples, use cross validation to find a good
set of hyper-parameters and benchmark on the test dataset another scikit-learn regression model ([see documentation](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)), for example choose between:
1. `sklearn.svm.SVR`: [Epsilon-Support Vector Regression](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)
2. `sklearn.neural_network.MLPRegressor` : [Multilayer Perceptron](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor)
3. **Advanced Track**: use `GradientBoostingRegressor` but add some engineering features in order to try to improve the previous result (e.g. moving averages and standard deviations of features)

In [ ]:
# space for the exercise solution

#### Recurrent Neural Network Model for Regression



### Predict Failures using Classification

**Will the unit fail within a certain time-frame (i.e. number of cycles)?**

This can be though of a classification problem,
where the boolean target is whether the unit will fail
within the next $n$ timesteps (e.g. 15 cycles).


#### Gradient Boosting for Classification

#### Exercise: Recurrent Neural Network Model for Classification


In [ ]:
# space for the exercise solution

## References

This notebook is heavily based on these resources on the topic:

- [1] [*Predictive Maintenance Template*](https://gallery.azure.ai/Collection/Predictive-Maintenance-Template-3)  by Microsoft Azure ML Team 
- [2] [*Predictive Maintenance using LSTM*](https://github.com/umbertogriffo/Predictive-Maintenance-using-LSTM) by Umberto Griffo
- [3] [*Predictive Maintenance ML (IOT)*](https://www.kaggle.com/billstuart/predictive-maintenance-ml-iiot) by Bill Stuart

The dataset used was provided by NASA:

- [4] A. Saxena and K. Goebel (2008). [*Turbofan Engine Degradation Simulation Data Set*](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan), NASA Ames Prognostics Data Repository, NASA Ames Research Center, Moffett Field, CA


Other resources and tutorials on Predictive Maintenance:

- [5] [*Predictive Maintenance Modelling Guide*](https://gallery.azure.ai/Collection/Predictive-Maintenance-Implementation-Guide-1) by Fidan Boyly Uz

Two good really good posts on the concept and usefulness of Recurrent Neural Networks for sequence data:

- [6] [*The Unreasonable Effectiveness of Recurrent Neural Networks*](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by Andrej Karpathy

- [7] [*Understanding LSTM Networks*](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) by Christopher Olah

This recent papers (with code) use this dataset in combination
with more advanced Deep Learning architectures and data augmentation
to achieve state of the art (SOTA):

- [8] S. Theng et al. [*Long short-term
memory network for remaining useful life estimation*](https://ieeexplore.ieee.org/document/7998311) in Proc. IEEE International Conference on Prognostics and Health

- [9] L. Jayasinghe et al. [*Temporal Convolutional Memory Networks for
Remaining Useful Life Estimation of Industrial Machinery*](https://github.com/LahiruJayasinghe/RUL-Net) in IEEE International Conference on Industrial Technology (ICIT2019)

Not many books on this topic, there is a book on ML for IOT (free 1 month subscription online), yet the
advanced ML chapter do not include specific IoT applications:

- [10] [Hands-On Artificial Intelligence for IoT](https://www.packtpub.com/big-data-and-business-intelligence/hands-artificial-intelligence-iot?utm_source=github&utm_medium=repository&utm_campaign=9781788836067) by Amita Kapoor (2019)